# Impact Investigation
1. Run all cells.
1. View report at the bottom.

In [ ]:
su = "tfs-wcus-0"
start = "2019-08-08T23:50:00.0000000Z"
end = "2019-08-09T00:24:36.0000000Z"
url = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK/delays.ipynb"
baseUrl = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK"
service = "tfs"
hub = "Build"
locationName = "tfsprodwcus0"

In [ ]:
%%capture
!pip install nimport azure-kusto-notebooks

In [ ]:
# Import the things we use

# Note you can also use kql https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic
# %kql is single line magic
# %%kql is cell magic

# https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb#HTML
# https://ipython.readthedocs.io/en/stable/inte/magics.html
from IPython.display import display, HTML, Markdown, Javascript, clear_output

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
import pandas as pd
pd.options.display.html.table_schema = True
from pandas import Series, DataFrame
from datetime import datetime, timedelta, timezone
from urllib.parse import urlencode, quote_plus
from requests.utils import requote_uri
import time
import numpy as np
from matplotlib import pyplot as plt
from nimport.utils import tokenize, open_nb
import json
import os
import calendar as cal
import concurrent.futures
from azure.kusto.notebooks import utils as akn

In [ ]:
params = {
    "su": su,
    "start": start,
    "end": end,
    "url": url,
    "baseUrl": baseUrl,
    "service": service
}
root = 'devops-pipelines' if os.path.basename(os.getcwd()) != 'devops-pipelines' else ''
queryPath = os.path.join(root, 'queries')

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net', 'VSO')

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
icm_client = akn.get_client('https://icmcluster.kusto.windows.net', 'IcMDataWarehouse')

In [ ]:
q_loc = os.path.join(queryPath, "LocationName.csl")
q_whatChanged = os.path.join(queryPath, "WhatChanged.csl")

impactPath = os.path.join(queryPath, "impact")
q_commands = os.path.join(impactPath, "CommandsReason.csl")
q_commandsAT = os.path.join(impactPath, "CommandsAT.csl")
q_commandsDb = os.path.join(impactPath, "CommandsDb.csl")
q_gen2 = os.path.join(impactPath, "Gen2GCSpikes.csl")
with concurrent.futures.ThreadPoolExecutor() as executor:
    # materialize location name immediately as we need this for other queries
    p1 = executor.submit(akn.execute_file, client, 'VSO', q_loc, params)
    locationNameResult = akn.to_dataframe_from_future(p1)
    locationName = locationNameResult["Tenant"][0]
    params["locationName"] = locationName
    p2 = executor.submit(akn.execute_file, client, 'VSO', q_whatChanged, params)
    
    p4 = executor.submit(akn.execute_file, client, 'VSO', q_commandsAT, params)
    p5 = executor.submit(akn.execute_file, client, 'VSO', q_commandsDb, params)        
    p6 = executor.submit(akn.execute_file, client, 'VSO', q_commands, params)
    
    p7 = executor.submit(akn.execute_file, icm_client, 'IcmDataWarehouse', 
                          os.path.join(queryPath, 'ActiveIncidents.csl'), params)
    
    p8 = executor.submit(akn.execute_file, client, 'VSO', q_gen2, params)

q_whatChanged_df = akn.to_dataframe_from_future(p2)

q_commandsAT_df = akn.to_dataframe_from_future(p4)

q_commandsDb_df = akn.to_dataframe_from_future(p5)

q_commands_df = akn.to_dataframe_from_future(p6)

q_activeIncidentsResultDf = akn.to_dataframe_from_future(p7)

q_gen2Df = akn.to_dataframe_from_future(p8)

In [ ]:
print('=' * 50)
print('Report!')
print('=' * 50, '\n\n')

# jarvis params
jarvisParams = {
  'su': su, 
  'start': akn.get_time(start, -10), 
  'end': akn.get_time(end, 10), 
  'service': service 
}

startTime = akn.to_datetime(start)
# jarvis
jarvisLink = """https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/DevOpsReports/TFS DevOpsReports""" \
    """?overrides=[{"query":"//*[id='Service']","key":"value","replacement":"%(service)s"},""" \
    """{"query":"//*[id='RoleInstance']","key":"value","replacement":""},""" \
    """{"query":"//*[id='ScaleUnit']","key":"value","replacement":"%(su)s"}]""" \
    """&globalStartTime=%(start)s&globalEndTime=%(end)s&pinGlobalTimeRange=true""" % jarvisParams;
print('Jarvis dashboard link:\n', requote_uri(jarvisLink), '\n')
    
# slow failed reason analysis
print()
print('Is it slow commands or failed commands? =============================')
freq = q_commands_df["Frequency"]
coefficientOfVariance = freq.std()/freq.mean()
failedCount = q_commands_df[q_commands_df["Reason"] == "failed"]["Frequency"].values[0]
slowCount = q_commands_df[q_commands_df["Reason"] == "slow"]["Frequency"].values[0]
reason = "failed or slow"
if coefficientOfVariance > 0.5:
    if failedCount > slowCount:
        reason = "failed"
    else:
        reason = "slow"
else:
    print("Slow and failed commands are too close, both might be contributing...")
if reason:
    print("Probably due to %s commands; Failed - %s, Slow - %s" % (reason, failedCount, slowCount))

# slow failed reason for AT?
print()
print('Is it %s because of AT? =============================' % (reason))
failed = q_commandsAT_df[q_commandsAT_df["Reason"] == "failed"]
slow = q_commandsAT_df[q_commandsAT_df["Reason"] == "slow"]
data = q_commandsAT_df
if reason == "failed":
    data = failed
elif reason == "slow":
    data = slow

coefficientOfVariance = data["Frequency"].std()/data["Frequency"].mean()
    
if coefficientOfVariance > 0.5:
    print("Found variance in AT's for %s commands" % (reason))
    print(data.head(30))
else:
    print("Seems be same across AT's for %s commands" % (reason))
    
# slow failed reason for Db?
print()
print('Is it %s because of Db? =============================' % (reason))
failed = q_commandsDb_df[q_commandsDb_df["Reason"] == "failed"]
slow = q_commandsDb_df[q_commandsDb_df["Reason"] == "slow"]
data = q_commandsDb_df
if reason == "failed":
    data = failed
elif reason == "slow":
    data = slow

coefficientOfVariance = data["Frequency"].std()/data["Frequency"].mean()
    
if coefficientOfVariance > 0.5:
    print("Found variance in Db's for %s commands" % (reason))
    print("Suffix '%s' to database server name" % (".database.windows.net"))
    print("Prefix '%s' to database name" % (params["service"] + "_" + params["locationName"] + "_"))
    print(data.head(30))
else:
    print("Seems be same across Db's for %s commands" % (reason))    
    
# what changed? analysis
print()
print('What changed? =============================')
if(len(q_whatChanged_df.index) == 0):
    print("...no relevant config changes recorded during this period.")
else:
    up_prefix = ""
    mit_prefix = ""
    vip_prefix = ""
    text = ""
    for index, row in q_whatChanged_df.iterrows():
        delta = startTime.replace(tzinfo=None) - row.TIMESTAMP.replace(tzinfo=None)
        hoursHappened = delta.total_seconds()//3600
        considerTime = hoursHappened <= 1
        def getText(row):
            return """%s %s %s (%s days %s hours %s minutes before the start time) \n\n""" % (row.TIMESTAMP, row.title, row.buildNumber, delta.days, delta.seconds//3600, delta.seconds//60)
        if(row.title.lower().find('upgrade') != -1):
            if not up_prefix:
                up_prefix += "Looks like, there's upgrade...\n\n"
            text += getText(row)
        if(row.title.lower().find('mitigation') != -1):
            if considerTime and not mit_prefix:
                mit_prefix += "Looks like, there are some mitigations by health agent...\n\n"
            text += getText(row)
        if(row.title.lower().find('vip') != -1):
            if considerTime and not mit_prefix:
                mit_prefix += "Looks like, there is VIP swap...\n\n"
            text += getText(row)            
    if text:
        print(up_prefix + mit_prefix + vip_prefix + text)
    else:
        print("...no relevant changes during this period.")
        
# active incidents?
print()
print('Active incidents? =============================')
otherIncidentsCount = 0;
for index, row in q_activeIncidentsResultDf.iterrows():
    if(row.Title.find("Customer Impact Monitor") == -1):
        otherIncidentsCount+=1;
        
if(otherIncidentsCount > 0):
    print("We found some incidents during the time period, check if they are related...")
    # styling
    def make_clickable(url, text):
        return '{0}'.format(url)

    newDf = q_activeIncidentsResultDf.assign(URL=[*map(lambda x: make_clickable("""https://icm.ad.msft.net/imp/v3/incidents/details/%s/home""" % (x), "ICMLink"), q_activeIncidentsResultDf.IncidentId)])
    print("ICM link to copy - " + "https://icm.ad.msft.net/imp/v3/incidents/details/INCIDENTID/home")
    print(newDf[['IncidentId','Severity','Title']])
else:
    print("No active incidents that could be related are found...")  
    

# additional insights
print()
print('Additional insights =============================')
w3wpGc = q_gen2Df[ q_gen2Df["ProcessName"] == "w3wp" ].sort_values(by=['Number'], ascending=False)
jaGc = q_gen2Df[ q_gen2Df["ProcessName"] == "TfsJobAgent" ].sort_values(by=['Number'], ascending=False)
now3wpGc = True
if len(w3wpGc) > 0:
    x = w3wpGc[:2]
    if len(x) == 1:
        xCount = x["Count"].values
        if xCount[0] > 25:
            print("INSIGHT: There's a new spike in w3wp gc...")
            now3wpGc = False
    if len(x) == 2:
        xCount = x["Count"].values
        if xCount[1] > xCount[0]:
            print("INSIGHT: There's a spike in w3wp gc...")
            now3wpGc = False
if now3wpGc:
    print("INSIGHT: No gc spikes found from w3wp...")
    
nojaGc = True            
if len(jaGc) > 0:
    x = jaGc[:2]
    if len(x) == 1:
        xCount = x["Count"].values
        if xCount[0] > 25:
            print("INSIGHT: There's a new spike in ja gc...")
            nojaGc = False
    if len(x) == 2:
        xCount = x["Count"].values
        if xCount[1] > xCount[0] and xCount[1] > 25:
            print("INSIGHT: There's a spike in ja gc...")  
            nojaGc = False
if nojaGc:
    print("INSIGHT: No ja gc spikes found from w3wp...")            